In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-136745", resource_group= "aml-quickstarts-136745", subscription_id= "610d6e37-4747-4a20-80eb-3aad70a55f43")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-136745
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-136745


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
cluster_name = "compute-cluster"
try:
    compute_target = ComputeTarget(workspace=ws, name="compute-cluster")
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive import BanditPolicy
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import normal, uniform, choice
from azureml.core import ScriptRunConfig
from azureml.core import Environment
import os

# Specify parameter sampler
parameter_sampler = RandomParameterSampling(parameter_space={"C": uniform(0.0, 1.0), 
                                 "max_iter": choice(50,100,150,200,250)})


# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.000000001, delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py. Used a ScriptRunConfig because SKLearn is deprecated
env_name = 'AzureML-Tutorial'
environment = Environment.get(workspace=ws, name=env_name)
script_run_config = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=environment
                      )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                             run_config=script_run_config,
                             hyperparameter_sampling=parameter_sampler,
                             policy=early_termination_policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=5,
                             max_concurrent_runs=2) 

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment

experiment = Experiment(ws, 'udacity-pipeline-optimization-experiment')
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
import joblib
# Get your best run and save the model from that run.

best_hyperdrive_run= hyperdrive_run.get_best_run_by_primary_metric()

print(best_hyperdrive_run.get_details())

Run(Experiment: udacity-pipeline-optimization-experiment,
Id: HD_e9995bd9-5c49-4a9a-b53f-af7dfa06a60a_1,
Type: azureml.scriptrun,
Status: Completed)
{'runId': 'HD_e9995bd9-5c49-4a9a-b53f-af7dfa06a60a_1', 'target': 'compute-cluster', 'status': 'Completed', 'startTimeUtc': '2021-01-31T16:38:52.565087Z', 'endTimeUtc': '2021-01-31T16:42:31.583917Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': 'e3455989-baed-483b-b2b5-3c7136a56ce7', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.5314884095444696', '--max_iter', '50'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'compute-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'pri

In [20]:
best_hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-pipeline-optimization-experiment,AutoML_435dd1c3-904b-4583-9f80-ddd3e326e82b_25,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(web_path)

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=2)

In [10]:
# Submit your automl run

automl_run = experiment.submit(automl_config, show_output=True)
automl_run.wait_for_completion()

Running on remote.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster
Parent Run ID: AutoML_435dd1c3-904b-4583-9f80-ddd3e326e82b

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead 

{'runId': 'AutoML_435dd1c3-904b-4583-9f80-ddd3e326e82b',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-31T16:52:44.10745Z',
 'endTimeUtc': '2021-01-31T17:39:42.688062Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-pipeline-optimization-experiment","subscription_id":"610d6e37-4747-4a20-80eb-3aad70a55f43","resource_group":"aml-quickstarts-136745","workspace_name":"quick-starts-ws-136745","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":2,"y_min":n

In [17]:
# Retrieve and save your best automl model.

automl_best_run, automl_best_model = automl_run.get_output()
print(automl_best_run.get_details())

{'runId': 'AutoML_435dd1c3-904b-4583-9f80-ddd3e326e82b_25', 'target': 'compute-cluster', 'status': 'Completed', 'startTimeUtc': '2021-01-31T17:36:22.583764Z', 'endTimeUtc': '2021-01-31T17:37:52.673666Z', 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-pipeline-optimization-experiment\',\'compute_target\':\'compute-cluster\',\'subscription_id\':\'610d6e37-4747-4a20-80eb-3aad70a55f43\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_435dd1c3-904b-4583-9f80-ddd3e326e82b_25","experiment_name":"udacity-pipeline-optimization-experiment","workspace_name":"quick-starts-ws-1

In [18]:
print(automl_best_run.get_metrics())

{'AUC_weighted': 0.945929865326163, 'f1_score_weighted': 0.9122194799059918, 'average_precision_score_weighted': 0.9546665744490062, 'AUC_macro': 0.9459298724955331, 'recall_score_micro': 0.9161153262518968, 'accuracy': 0.9161153262518968, 'average_precision_score_macro': 0.8226553691623013, 'precision_score_micro': 0.9161153262518968, 'matthews_correlation': 0.5440951199705462, 'f1_score_macro': 0.7699058800110591, 'recall_score_macro': 0.7478018521208536, 'AUC_micro': 0.9803313504390014, 'log_loss': 0.20237736438749693, 'norm_macro_recall': 0.4956037042417073, 'precision_score_macro': 0.7986730291523929, 'f1_score_micro': 0.9161153262518968, 'precision_score_weighted': 0.9100237360933093, 'balanced_accuracy': 0.7478018521208536, 'recall_score_weighted': 0.9161153262518968, 'weighted_accuracy': 0.9579273725240036, 'average_precision_score_micro': 0.9810163274551058, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_435dd1c3-904b-4583-9f80-ddd3e326e82b_25/accuracy_table', '

In [19]:
automl_best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-pipeline-optimization-experiment,AutoML_435dd1c3-904b-4583-9f80-ddd3e326e82b_25,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
#delete compute cluster
comupute_cluster.delete()